<a href="https://colab.research.google.com/github/Heisnotanimposter/UPbit-quant/blob/main/KRWBTC_crawler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyupbit

import time
import pyupbit
import datetime


access_key = " "
secret_key = " "
upbit = pyupbit.Upbit(access_key, secret_key)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Define your trading strategy
def buy_sell_strategy(ticker):
    # Get current ticker price and MACD indicator
    df = pyupbit.get_ohlcv(ticker)
    macd = df['close'].ewm(span=12).mean() - df['close'].ewm(span=26).mean()
    macd_signal = macd.ewm(span=9).mean()
    current_price = pyupbit.get_current_price(ticker)

    # Make buy and sell decisions based on the MACD indicator
    if macd[-2] < macd_signal[-2] and macd[-1] > macd_signal[-1]:
        krw_balance = upbit.get_balance("KRW")
        if krw_balance > 10000:
            upbit.buy_market_order(ticker, krw_balance*0.9995)
            return True
    elif macd[-2] > macd_signal[-2] and macd[-1] < macd_signal[-1]:
        coin_balance = upbit.get_balance(ticker)
        if coin_balance > 0.0001:
            upbit.sell_market_order(ticker, coin_balance)
            return True
    return False


In [ ]:
# Run auto quant trading
for i in range(1000):
    try:
        now = datetime.datetime.now()
        start_time = datetime.datetime(now.year, now.month, now.day)  # Start of the day

        # Get the candlestick data for the KRW-BTC market
        df = pyupbit.get_ohlcv("KRW-BTC", interval="day", to=now)

        # Save the candlestick data to a CSV file
        df.to_csv('krw_btc_candlesticks.csv')

        # Get the start time of the first candlestick
        if start_time == df.index[0]:
            if buy_sell_strategy("KRW-BTC"):
                print("Trading signal detected and executed.")

        # Wait for the next trading opportunity
        time.sleep(1)
    except Exception as e:
        print(e)
        time.sleep(1)

KeyboardInterrupt: ignored